In [0]:
#A marketing agency noticed that they have quite a bit of churn in clients
#We want to predict wich customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churm an accoutn manager
#Create a classification algorithm that will help classify wether or not a customer churned
#Then the company can test this against icoming data for future customers to predict wich customers will churn and assign them to an account manager

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregConsul').getOrCreate()

In [0]:
data = spark.read.csv('dbfs:/FileStore/shared_uploads/gkantirisrafael@gmail.com/customer_churn.csv',header=True,inferSchema=True)

In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [0]:
data.columns

Out[5]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
#This is our final data set using onlythe columns we need
final_data = output.select('features','churn')

In [0]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
lr_churn = LogisticRegression(labelCol='churn')

In [0]:
#We now will fit this model into our data
fitted_churn_model = lr_churn.fit(train_churn)

In [0]:
train_sum = fitted_churn_model.summary

In [0]:
train_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                620|                620|
|   mean|0.16451612903225807|0.12419354838709677|
| stddev| 0.3710426187668644|0.33006852173066925|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [0]:
pred_and_labels.predictions.show(3)

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.61429914546674...|[0.99636787673749...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.10189988597237...|[0.99969712876297...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.39537846770878...|[0.80144949687118...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [0]:
#Area Under Curve
AUC = churn_eval.evaluate(pred_and_labels.predictions)

In [0]:
AUC
#Aera under curve

Out[21]: 0.7636494252873564

In [0]:
# Now we will try to predict under new data
final_lr_model = lr_churn.fit(final_data)

In [0]:
new_customers = spark.read.csv('dbfs:/FileStore/shared_uploads/gkantirisrafael@gmail.com/new_customers.csv',inferSchema=True,header=True)

In [0]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
final_results = final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [0]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          